# Prediction of Next Year's Severe Malnutrition Rate for each District in Sierra Leone

**Data** Severe malnutrition data for the past 10 years for each district of Sierra Leone was downloaded. Data ranges from 2014 to 2023. 2024 data is incomplete.

**Methodology**
- Data was checked to see if there are any missing values
- Data was scaled using min max scalar to avoid any potential bias
- For this analysis ARIMA model was chosen becuase it was one of the easiest timeseries models to use for prediction of next year's data.
- Default parameters were used and model was fit for each district.
- After training, predictions were made for the next year.

**Considerations**
- The amount of data available is short and therefore predictions must be taken with caution
- The Mean Absolute Error (MAE) for this prediction was not low engough suggesting that model is not that accurate.

**Rate to population convertion** To convert the severe malnutrition rate to the population, for each district, we need total number of children. This was not available so I used some example total population data to get the numbers.


In [1]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import MinMaxScaler

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
data = pd.read_csv('/content/drive/MyDrive/SL_SM_10yr.csv', header=0)

# Set the 'Year' column as the index
data.set_index('Year', inplace=True)
data.head()

,Bo,Bombali,Bonthe,Kailahun,Kambia,Kenema,Koinadugu,Kono,Moyamba,Port Loko,Pujehun,Tonkolili,Western Area
Year,,,,,,,,,,,,,
2014,5.49,2.50,5.64,3.37,6.79,5.44,8.07,9.64,13.28,5.47,2.36,7.80,7.06
2015,5.53,2.55,5.59,3.34,6.80,5.37,8.02,9.49,13.33,5.39,2.34,8.00,7.17
2016,5.49,2.50,5.49,3.33,6.73,5.37,8.01,9.22,13.23,5.46,2.38,8.10,7.06
2017,5.51,2.51,5.49,3.30,6.75,5.44,7.99,9.38,13.26,5.34,2.34,7.86,6.99
2018,5.50,2.58,5.68,3.37,6.80,5.40,8.17,9.55,13.25,5.44,2.36,7.80,7.04


## ARIMA Model
- First scale the data using minmax scaler.
- at the end, reconvert the scaled data to normal data to get the ratee
- in the model, use default parameters for order as 1, 1, 1.
- To get optimized results, it is necessary to perform parameter tuning for all posible combinations of order. This was not done at this time.
- Fit the model to the data, forecast the model for next year
-

In [4]:
# Scale the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# Convert scaled data back to DataFrame for easier handling
data_scaled = pd.DataFrame(data_scaled, columns=data.columns, index=data.index)

In [5]:
# Write a function that would take each district scaled data, fit the model, forecast the model and inverse scale the data.

def forecast_for_districts(district_data, district_data_scaled, scaler):
    forecast_dict = {}

    # Iterate over each district in the original DataFrame to maintain column names
    for idx, district in enumerate(district_data.columns):
        time_series = district_data[district]

        # Fit ARIMA model (use (p, d, q) = (1, 1, 1) as an example)
        model = ARIMA(time_series, order=(1, 1, 1))
        model_fit = model.fit()

        # Forecast the next year (1 year = 1 step ahead per ARIMA)
        forecast = model_fit.forecast(steps=1)

        # Inverse the scaling to get the original value
        # Create a temporary array with the same shape as the original data
        # Instead of -1, use the last valid index label
        temp_scaled = district_data_scaled.iloc[-1].copy()
        temp_scaled[idx] = forecast.values[0]
        forecast_unscaled = scaler.inverse_transform([temp_scaled])[0][idx]

        # Store the forecasted value in the dictionary
        forecast_dict[district] = forecast_unscaled

    # Return the forecasted values for each district
    return forecast_dict

In [6]:
# Forecast the next year's data for each district
forecast_next_year = forecast_for_districts(data, data_scaled, scaler)

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be give

In [7]:
# Display the forecasted values for the next year
forecast_df = pd.DataFrame(list(forecast_next_year.items()), columns=['District', 'Forecast for 2024'])
# Display the forecast table
print(forecast_df)

        District  Forecast for 2024
0             Bo           5.990670
1        Bombali           2.704063
2         Bonthe           6.552037
3       Kailahun           3.534968
4         Kambia           7.550167
5         Kenema           5.749019
6      Koinadugu           9.648047
7           Kono          13.470898
8        Moyamba          15.424295
9      Port Loko           6.044507
10       Pujehun           2.626410
11     Tonkolili          10.234970
12  Western Area           8.267562


## In the above tale, the forecasted severe malnutrition rate for 2024 was given. Convert that to population.

In [8]:
# Example population data for each district
population_data = {
    'Bo': 100000,
    'Bombali': 80000,
    'Bonthe': 60000,
    'Kailahun': 70000,
    'Kambia': 50000,
    'Kenema': 90000,
    'Koinadugu': 40000,
    'Kono': 75000,
    'Moyamba': 65000,
    'Port Loko': 85000,
    'Pujehun': 55000,
    'Tonkolili': 80000,
    'Western Area': 120000
}

# Calculate the number of affected individuals for each district
affected_population = {district: int(forecast_next_year[district] * population_data[district]) for district in forecast_next_year}

# Display the affected population without scientific notation
affected_population_df = pd.DataFrame(list(affected_population.items()), columns=['District', 'Affected Population in 2024'])
pd.set_option('display.float_format', '{:.0f}'.format)
print(affected_population_df)

        District  Affected Population in 2024
0             Bo                       599067
1        Bombali                       216325
2         Bonthe                       393122
3       Kailahun                       247447
4         Kambia                       377508
5         Kenema                       517411
6      Koinadugu                       385921
7           Kono                      1010317
8        Moyamba                      1002579
9      Port Loko                       513783
10       Pujehun                       144452
11     Tonkolili                       818797
12  Western Area                       992107
